In [1]:
import nltk
from nltk import trigrams
from nltk.corpus import brown
from collections import defaultdict

In [2]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
uniModel = defaultdict(lambda: 0)
biModel = defaultdict(lambda: defaultdict(lambda: 0))
triModel = defaultdict(lambda: defaultdict(lambda: 0))
model1 = defaultdict(lambda: set())
model2 = defaultdict(lambda: set())

In [4]:
def convert_to_lower(c):
  if type(c)==str:
    return c.lower()
  return c

In [5]:
totalWord = 0
for sentence in brown.sents():
  for word in sentence:
    uniModel[word] += 1
    totalWord +=1

  for firstWord, secondWord, thirdWord in trigrams(sentence, pad_right=True, pad_left=True):
    firstWord = convert_to_lower(firstWord)
    secondWord = convert_to_lower(secondWord)
    thirdWord = convert_to_lower(thirdWord)

    biModel[thirdWord][secondWord] += 1
    triModel[thirdWord][firstWord] += 1

    model1[secondWord].add(thirdWord)
    model2[firstWord].add(thirdWord)

In [6]:
def calculate_probablities(model):
  for nextWord in model:
    totalCount = float(sum(model[nextWord].values()))
    for prevWord in model[nextWord]:
        model[nextWord][prevWord] /= totalCount

calculate_probablities(biModel)
calculate_probablities(triModel)

for word in uniModel:
  uniModel[word] /= totalWord

In [7]:
def calculate_trigram_Probability(firstWord, secondWord, thirdWord):
  return uniModel[thirdWord]*biModel[thirdWord][secondWord]*triModel[thirdWord][firstWord]

In [8]:
def predict_word(firstWord,secondWord):
  response = []
  for thirdWord in model1[secondWord] & model2[firstWord]:
    res = calculate_trigram_Probability(firstWord,secondWord,thirdWord)
    response.append((thirdWord,res))
  return response

In [ ]:
while(True):
    text = input("Enter text : ")
    if text == "STOP":
        break
    
    else:
        try:
            text = text.split(" ")
            data = predict_word(text[-2], text[-1])
            sorted(data, key=lambda o: o[1], reverse=True)
            print("---> SUGGESTIONS <---")
            print(data[:10])
            print()
            print()
            
        except:
            continue

Enter a line (In lower charecter): i live in
################## SUGGESTIONS ##################
[('accordance', 8.611840246918683e-07), ('modern', 3.954012969200499e-08), ('pasadena', 0.0), ('is', 8.436397477790134e-11), ('missoula', 0.0), ('it', 1.652323115868559e-08), ('?', 1.8350394730276335e-09), (';', 3.094444932417781e-10), ('.', 5.933663688956252e-09), ('trade', 3.950269524969302e-08)]


Enter a line (In lower charecter): what's on your
################## SUGGESTIONS ##################
[('heels', 2.39308226143109e-07), ('music', 7.16176700918306e-09), ('new', 4.097769845067499e-09), ('tour', 1.336721552658552e-07), ('body', 3.041095655352245e-09), ('cheek', 2.1529600617296709e-07), ('sick', 1.6223766708920243e-08), ('entrance', 1.4843430404045745e-08), ('problem', 1.645557772584366e-08), ('house', 1.380730855178055e-08)]


Enter a line (In lower charecter): bangladesh is a
################## SUGGESTIONS ##################
[('music', 4.297060205509836e-08), ('poor', 2.978297950535